# Advanced RAG System Demo

### Objectives:
1. **System Resource Monitoring**: Track RAM/CPU usage during heavy tasks.
2. **Pipeline Execution**: Load data, chunk, index, retrieve, and generate.
3. **Evaluation**: Compare Generated Answers vs Reference Answers using **BLEU-4** and **ROUGE-L** metrics.

In [ ]:
# Setup Environment & Utils
!pip install -r requirements.txt

import sys
import os
import psutil

# Ensure src is in python path
sys.path.append(os.getcwd())

def print_system_usage(stage=""):
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    mem_mb = mem_info.rss / 1024 / 1024
    print(f"[{stage}] Memory: {mem_mb:.2f} MB")

In [ ]:
# Load Configuration & Modules
from src.rag.config import RAGConfig
from src.rag.data_loader import DataLoader
from src.rag.chunking import HierarchicalChunker
from src.rag.vector_db import VectorDBHandler
from src.rag.retriever import HierarchicalRetriever
from src.rag.generator import RAGGenerator
from src.rag.evaluator import Evaluator

config = RAGConfig()
print("Configuration Loaded.")
print_system_usage("Init")

In [ ]:
# Authenticate with Hugging Face (Required for Gemma Model)
from huggingface_hub import login

print("Please paste your Hugging Face Token when prompted.")
print("You can find it here: https://huggingface.co/settings/tokens")
# If you have already logged in via terminal, you can comment this out.
login()

In [ ]:
# Data Loading
loader = DataLoader(config)

# Download Book
book_text = loader.download_book()
print(f"Book loaded. Length: {len(book_text)} chars")

# Load QA Pairs
qa_pairs = loader.load_qa_pairs()
print(f"Loaded {len(qa_pairs)} QA pairs for testing.")
print_system_usage("Data Loading")

In [ ]:
# Hierarchical Chunking
chunker = HierarchicalChunker(
    parent_chunk_size=config.PARENT_CHUNK_SIZE,
    child_chunk_size=config.CHILD_CHUNK_SIZE,
    overlap=config.CHUNK_OVERLAP
)

chunks = chunker.chunk_data(book_text)
print(f"Created {len(chunks['parents'])} parent chunks and {len(chunks['children'])} child chunks.")

parents = chunks['parents']
children = chunks['children']
print_system_usage("Chunking")

In [ ]:
# Indexing in Qdrant (Local Disk Mode)
# CRITICAL: Force cleanup of previous instances to release file locks
import gc
try:
    if 'vdb' in locals():
        print("Cleaning up previous DB instance...")
        if hasattr(vdb, 'close'):
            vdb.close()
        del vdb
        gc.collect() # Force garbage collection to release file handles
except Exception as e:
    print(f"Cleanup warning: {e}")

vdb = VectorDBHandler(config)
vdb.create_collection()

print("Indexing chunks... (this creates embeddings using CPU/GPU)")
vdb.index_chunks(chunks)
print_system_usage("Indexing")

In [ ]:
# Initialize Components
retriever = HierarchicalRetriever(config, vdb, parents)
generator = RAGGenerator(config)
evaluator = Evaluator()
print("RAG Components Ready.")
print_system_usage("Model Load")

In [ ]:
# Run RAG Loop & Evaluation
import pandas as pd

# Run on a subset or all pairs
test_pairs = qa_pairs[:5] # Testing on first 5 pairs for demo speed
results = []

print(f"Running RAG on {len(test_pairs)} queries...")

for i, qa in enumerate(test_pairs):
    question = qa['question']
    reference = qa['answer1']
    
    # 1. Retrieve
    context = retriever.retrieve_context(question, top_k=config.TOP_K)
    
    # 2. Generate
    generated_answer = generator.generate_answer(question, context)
    
    # 3. Evaluate
    scores = evaluator.evaluate(generated_answer, reference)
    
    results.append({
        "Question": question,
        "Generated Answer": generated_answer,
        "Reference Answer": reference,
        "BLEU-4": scores['bleu'],
        "ROUGE-L": scores['rouge']
    })
    print(f".", end="") # Progress indicator

print("\nDone!")
print_system_usage("Inference Complete")

In [ ]:
# Results Analysis
df_results = pd.DataFrame(results)

# Calculate Averages
avg_bleu = df_results['BLEU-4'].mean()
avg_rouge = df_results['ROUGE-L'].mean()

print("--- Evaluation Summary ---")
print(f"Average BLEU-4: {avg_bleu:.4f}")
print(f"Average ROUGE-L: {avg_rouge:.4f}")

# Display Table
df_results[['Question', 'Generated Answer', 'BLEU-4', 'ROUGE-L']]